<a href="https://colab.research.google.com/github/mr-ankit-tech/Coding_with_llm/blob/Workspace/Super_Agent_for_getting_insight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# REQUIRED INSTALLS (if not already done)
!pip install langchain langchain_openai langchain_community
!pip install transformers accelerate bitsandbytes
!pip install huggingface_hub
!pip install langchain-huggingface

# === Python Imports ===
import os
import sqlite3
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.prompts import PromptTemplate
from google.colab import userdata, data_table
from IPython.display import display
from datetime import datetime # Import datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [3]:
# === Load Data into SQLite ===
daily_data_df = pd.read_csv("/content/Heartbeat.csv")
sqlite_connection = sqlite3.connect("ankit_tutorial.db")
db_path = os.path.abspath("ankit_tutorial.db")
daily_data_df.to_sql("Heartbeat", sqlite_connection, if_exists="replace")

daily_data_df = pd.read_csv("/content/Audit.csv")
sqlite_connection = sqlite3.connect("ankit_tutorial.db")
db_path = os.path.abspath("ankit_tutorial.db")
daily_data_df.to_sql("Audit", sqlite_connection, if_exists="replace")


4013

In [4]:

# === Initialize ChatOpenAI ===
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-1106")

# === Create SQLDatabase Object ===
my_db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
print(f"Tables available to the agent: {my_db.get_table_names()}")

# === Utility: Dynamic Schema Fetch for a Given Table ===
def get_table_schema(table_name: str) -> str:
  db=my_db
  if table_name not in db.get_table_names():
    raise ValueError(f"Table '{table_name}' does not exist in the database.")
  # Pass the table name as a list to get_table_info
  return db.get_table_info([table_name])

# === Prompt Template to Generate SQL ===
sql_prompt_template = PromptTemplate(
    input_variables=["user_inquiry", "background_info"],
    template="""
    You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run.
    Never query all columns (*).
    Only use the columns you see in the schema below.

    {background_info}
    Question: {user_inquiry}

    Only return the SQL query, nothing else.
    """
    )

# === Full Function to Handle Dynamic Table & Question ===
def Super_Agent(table_name: str, user_inquiry: str) -> pd.DataFrame:
    # Get schema for selected table
    table_schema = get_table_schema(table_name)
    background_info = f"""
    You are a SQLite expert. The following is the schema of the table `{table_name}`.
    Use only the columns listed below when writing SQL queries.
    Never use SELECT *.
    {table_schema}
    """


    # Format prompt
    formatted_prompt = sql_prompt_template.format(
        background_info=background_info,
        user_inquiry=user_inquiry
    )

    # Generate SQL from LLM
    sql_query = llm.invoke(formatted_prompt).content.strip()
    print("Generated SQL query:")
    print(sql_query)

    # Execute SQL
    conn = sqlite3.connect(db_path)
    df_result = pd.read_sql_query(sql_query, conn)
    conn.close()

    # Show Data
    data_table.enable_dataframe_formatter()
    display(df_result)

 # === Generate dynamic CSV filename ===
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    safe_table_name = table_name.replace(" ", "_").lower()
    filename = f"Extract_{safe_table_name}_{timestamp}.csv"

    # Save to CSV
    df_result.to_csv(filename, index=False)
    print(f"✅ Data saved to {filename}")

    #return df_result

/tmp/ipython-input-4-4082961361.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-1106")


Tables available to the agent: ['Audit', 'Heartbeat']


/tmp/ipython-input-4-4082961361.py:7: LangChainDeprecationWarning: The method `SQLDatabase.get_table_names` was deprecated in langchain-community 0.0.1 and will be removed in 1.0. Use :meth:`~get_usable_table_names` instead.
  print(f"Tables available to the agent: {my_db.get_table_names()}")


In [5]:
# You can dynamically change table name and question here
Super_Agent("Heartbeat","Compare actual revenue and booked revenue by Trade")
Super_Agent("Audit","Can you please share the amount by countries")

# Other Examples:
# query_table(llm, my_db, "Insight", "What is the total Actual_FFE by Year?")
# query_table(llm, my_db, "Insight", "List all shipments from Loading_country = 'India'")


Generated SQL query:
SELECT Trade, SUM(Actual_Revenue) AS Total_Actual_Revenue, SUM(Booked_Revenue) AS Total_Booked_Revenue
FROM Heartbeat
GROUP BY Trade;


,Trade,Total_Actual_Revenue,Total_Booked_Revenue
0,B3 - WCSA - North America,3.777300e+04,3.513849e+05
1,C2 - North America-Central America,8.137914e+05,5.712081e+06
2,I1 - IET,2.122585e+05,1.617782e+06
3,L7 - Intra NEU,2.342156e+06,2.477317e+07
4,R1 - Intra Asia,3.572907e+05,1.332669e+06
5,U2 - Intra Americas,3.964662e+04,2.792303e+05
6,UG - ECSA - ECSA,1.290588e+07,1.151516e+08
7,W5 - Intra Africa,3.824391e+07,3.900577e+08
8,X1 - Europe - French Antilles,4.072264e+04,2.545165e+05
9,Z1 - Americas - East Africa,6.792327e+07,4.147457e+08


✅ Data saved to Extract_heartbeat_20250622_081339.csv
Generated SQL query:
SELECT Country, SUM(Amount) AS Total_Amount
FROM Audit
GROUP BY Country;


,Country,Total_Amount
0,None,3.898785e+09
1,Australia,1.501765e+06
2,Belgium,2.339197e+06
3,Benin,1.184232e+05
4,Brazil,1.791583e+06
5,Cambodia,-7.857669e+04
6,Cameroon,9.201448e+04
7,Canada,1.534580e+07
8,Chile,-2.300000e+00
9,China,2.938684e+07


✅ Data saved to Extract_audit_20250622_081339.csv
